<a href="https://colab.research.google.com/github/xiekg2020/DL_test/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.3430416 ],
       [-0.46470043]], dtype=float32)>

In [3]:
print(net.layers[2].weights)

[<tf.Variable 'dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.25660628],
       [ 0.05566728],
       [ 1.0512323 ],
       [-1.0079228 ]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [4]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [5]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.24299997,  0.648976  ,  0.5335247 ,  0.54134697],
       [-0.11794209,  0.26983434,  0.60853094,  0.85462373],
       [ 0.30730087, -0.45540857, -0.09013319, -0.39574534],
       [-0.4047506 ,  0.7234542 ,  0.3493021 ,  0.8482755 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.24299997,  0.648976  ,  0.5335247 ,  0.54134697],
       [-0.11794209,  0.26983434,  0.60853094,  0.85462373],
       [ 0.30730087, -0.45540857, -0.09013319, -0.39574534],
       [-0.4047506 ,  0.7234542 ,  0.3493021 ,  0.8482755 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.25660628],
       [ 0.05566728],
       [ 1.0512323 ],
       [-1.0079228 ]], dtype=float32), array([0.], dtype=float32)]


In [6]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [7]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)], name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.58385164],
       [-0.4890989 ]], dtype=float32)>

In [8]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (2, 4)                    80        
_________________________________________________________________
dense_6 (Dense)              (2, 1)                    5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_7/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00444682,  0.00551663,  0.00932786, -0.01207432],
        [ 0.00023069, -0.00575974,  0.00981569,  0.00199213],
        [ 0.00553036, -0.00109793,  0.00605413,  0.00272769],
        [ 0.00926816, -0.00186185, -0.01348842, -0.0189439 ]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [12]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data = tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor = (tf.abs(data) >= 5)
        factor = tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu,
                          kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-9.008596 ,  7.198681 ,  0.       ,  0.       ],
       [-0.       , -7.7233124,  0.       ,  0.       ],
       [-0.       , -0.       , -0.       ,  0.       ],
       [-0.       ,  0.       , -0.       ,  9.306889 ]], dtype=float32)>


In [13]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  8.198681 ,  1.       ,  1.       ],
       [ 1.       , -6.7233124,  1.       ,  1.       ],
       [ 1.       ,  1.       ,  1.       ,  1.       ],
       [ 1.       ,  1.       ,  1.       , 10.306889 ]], dtype=float32)>

In [14]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(), shared, shared,
    tf.keras.layers.Dense(1),])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True
